# 浪子之心科技
## wav2lip_288x288训练操作手册

In [ ]:
#@title 一、下载源码安装依赖
git clone https://github.com/primepake/wav2lip_288x288

In [1]:
#@title 二、下载源码安装依赖
# 这里需要提前将人脸检测器的权重提前上传到网盘，再进行下载，我这边用的是我自己的分享链接，后期可能会关闭。
cd ./wav2lip_288x288
# 下载人脸检测器模型
wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/root/wav2lip_288x288/face_detection/detection/sfd/s3fd.pth"

In [ ]:
# #@title 三、安装依赖
# cd wav2lip_288x288
# a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
# # !pip uninstall tensorflow tensorflow-gpu
# pip install librosa numpy opencv-contrib-python opencv-python torch torchvision tqdm numba
# pip install -q youtube-dl
# pip install ffmpeg-python
# pip install librosa==0.9.1

In [ ]:
#@title 四、进行数据预处理
# 上传收集好的视频到source_data，要求说话清晰，嘴型清晰，没有杂音没有背景音。
# 25帧率，16000赫兹，每个人说话1，2分钟。几千人效果较好。我这个模型是600人的，20多个小时的数据集。

# 将数据切割成5秒左右的说话视频
python split_video.py
# 生成data下的文件

# 生成filelists文件夹下面的train.txt val.txt
python sample.py

In [ ]:
# 数据处理
# 如果爆显存 请将 preprocess.py 的 batch_size 改成16 或者更小。
# 如果爆显存 请将 preprocess.py 的 batch_size 改成8 或者更小。
cd wav2lip_288x288
python preprocess.py --data_root ./data --preprocessed_root ./preprocessed_data --batch_size 4

In [ ]:
#@title 三、训练判别器
# 高清的判别器需要自己训练 lipsync_expert.pth, 大概227M大小。
# 如果爆显存 ，请将 hparams.py的 batch_size 改成4 或者更小。

cd wav2lip_288x288
mkdir expert_checkpoints

python color_syncnet_train.py --data_root ./preprocessed_data --checkpoint_dir ./expert_checkpoints
# python color_syncnet_train.py --data_root ./preprocessed_data --checkpoint_dir ./expert_checkpoints --checkpoint_path ./expert_checkpoints/checkpoint_step000135000.pth


In [ ]:
#@title 四、训练生成器
# 如果爆显存 ，请将 hparams.py 的 batch_size 改成4 或者更小。
cd wav2lip_288x288
python wav2lip_train.py --data_root ./preprocessed_data --checkpoint_dir ./checkpoints --syncnet_checkpoint_path ./expert_checkpoints/checkpoint_step000005000.pth

In [ ]:
#@title 五、进行推理
cd wav2lip_288x288
python inference.py --checkpoint_path ./checkpoints/checkpoint_step000387000.pth --face ./input/555.mp4 --audio ./input/555.wav

# 测试
python inference.py --checkpoint_path checkpoints/checkpoint_step000387000.pth --face input/video000.mp4 --audio input/mingzheng-clone-16k.wav --outfile results/127.mp4